In [ ]:
from pykrx import stock
from tqdm import tqdm
import pandas as pd
import FinanceDataReader as fdr

stocks = fdr.StockListing('KRX')['Symbol']

def date_to_str(df) :
    tmp = pd.DataFrame(df.index)
    print(len(tmp))
    for i in range(len(tmp)) :
        tmp.loc[i,'날짜'] = str(tmp.loc[i,'날짜'])[0:4]+str(tmp.loc[i,'날짜'])[5:7]+str(tmp.loc[i,'날짜'])[8:10]
    return tmp

def start_end_date(code) :
    try : 
        print(code)
        df2 = fdr.DataReader(code)
    except : 
        print(code)
        df2 = fdr.DataReader(code, exchange='KRX-DELISTING')
    if len(df2) == 0:
        print('0')
        return 0
    start_date = str(df2.index[0])[0:4]+str(df2.index[0])[5:7]+str(df2.index[0])[8:10]
    end_date = str(df2.index[-1])[0:4]+str(df2.index[-1])[5:7]+str(df2.index[-1])[8:10]
    if int(start_date) < int('20010101') :
        start_date = '20010101'
    return [start_date, end_date]

In [ ]:
# 삼성전자, SK하이닉스로 2001.01.01~2021.10.11까지의 유효거래일 DF 생성
# 5133일로 동일
sk_df = stock.get_market_ohlcv_by_date("20010101", "20211011", "000660") #20211011
sm_df = stock.get_market_ohlcv_by_date("20010101", "20211011", "005930")
sk = date_to_str(sk_df)
sm = date_to_str(sm_df)
date_df = pd.merge(sm, sk, left_on='날짜', right_on='날짜', how='outer')
date_df_stamp = sk_df.index
print(sk_df.index)

In [ ]:
kd = pd.read_csv ('G:\\공유 드라이브\\WARD\\Stock_Price\\SP_o_kd.csv', dtype=object)
kp = pd.read_csv ('G:\\공유 드라이브\\WARD\\Stock_Price\\SP_o_kp.csv', dtype=object)

for i in range(len(kd['티커'])) :
    kd.loc[i, '티커'] = str(kd['티커'][i]).zfill(6)
               
for i in range(len(kp['티커'])) :
    kp.loc[i, '티커'] = str(kp['티커'][i]).zfill(6)
    
kp

In [ ]:
# KOSPI - Short

Short_v =  pd.DataFrame(index = date_df_stamp) # 공매도
Short_l = pd.DataFrame(index = date_df_stamp)  # 공매도 잔고
Short_vc = pd.DataFrame(index = date_df_stamp) # 공매도 금액
Short_lc = pd.DataFrame(index = date_df_stamp) # 공매도 잔고 금액

i = 0
code = kp['티커'][i]
date = start_end_date(code)
df = stock.get_shorting_status_by_date(date[0], date[1], code)

df_v = df['거래량'].rename(code)
df_l = df['잔고수량'].rename(code)
df_vc = df['거래대금'].rename(code)
df_lc = df['잔고금액'].rename(code)

Short_v = pd.merge(Short_v, df_v , left_on='날짜', right_on='날짜', how='outer')
Short_l = pd.merge(Short_l, df_l , left_on='날짜', right_on='날짜', how='outer')
Short_vc = pd.merge(Short_vc, df_vc , left_on='날짜', right_on='날짜', how='outer')
Short_lc = pd.merge(Short_lc, df_lc , left_on='날짜', right_on='날짜', how='outer')

for i in tqdm(range(len(kp['티커']))):
    code = kp['티커'][i]
    date = start_end_date(code)
    if date == 0 :
        continue
    try :
        df = stock.get_shorting_status_by_date(date[0], date[1], code)
    except :
        print("pass")
    df_v = df['거래량'].rename(code)
    df_l = df['잔고수량'].rename(code)
    df_vc = df['거래대금'].rename(code)
    df_lc = df['잔고금액'].rename(code)

    Short_v = pd.merge(Short_v, df_v , left_on='날짜', right_on='날짜', how='outer')
    Short_l = pd.merge(Short_l, df_l , left_on='날짜', right_on='날짜', how='outer')
    Short_vc = pd.merge(Short_vc, df_vc , left_on='날짜', right_on='날짜', how='outer')
    Short_lc = pd.merge(Short_lc, df_lc , left_on='날짜', right_on='날짜', how='outer')
    
Short_v =  Short_v.transpose()
Short_l = Short_l.transpose()
Short_vc = Short_vc.transpose()
Short_lc = Short_lc.transpose()

Short_v.to_csv('Short_v_kp.csv', index = True, header = True)
Short_l.to_csv('Short_l_kp.csv', index = True, header = True)
Short_vc.to_csv('Short_vc_kp.csv', index = True, header = True)
Short_lc.to_csv('Short_lc_kp.csv', index = True, header = True)